In [9]:
# -*- coding: utf-8 -*-
"""
Hückel com PBC (cadeia C–N, 2 sítios/célula) — GERA FIGURAS E DADOS (sem LaTeX).
Arquivos gerados:
  fig1_matrizes.png, fig2_bandas.png, fig3_populacoes.png, fig4_ordens.png
  bandas.csv, observaveis.json
"""

from __future__ import annotations
import json
from typing import Tuple
import numpy as np
import matplotlib.pyplot as plt

# ---------------- Parâmetros do modelo ----------------
alfa_C: float = 0.0       # eV
alfa_N: float = -1.0      # eV
beta: float = -2.5        # eV  # ligação intra-célula C–N
beta_linha: float = -2.0  # eV  # ligação inter-célula N_n – C_{n+1}
a: float = 1.0            # unidade de rede

# ---------------- Hamiltoniano ----------------
def hamiltoniano_k(k: float) -> np.ndarray:
    """Hamiltoniano 2x2 no espaço recíproco (base |C>, |N>)."""
    off = beta + beta_linha * np.exp(-1j * k * a)
    return np.array([[alfa_C, off],
                     [np.conjugate(off), alfa_N]], dtype=complex)

def obter_matrizes_gamma_x() -> Tuple[np.ndarray, np.ndarray]:
    """Retorna H(Γ) e H(X) explícitos (reais)."""
    H_gamma = np.array([[alfa_C, beta + beta_linha],
                        [beta + beta_linha, alfa_N]], dtype=float)
    H_x = np.array([[alfa_C, beta - beta_linha],
                    [beta - beta_linha, alfa_N]], dtype=float)
    return H_gamma, H_x

# ---------------- Bandas e autovetores ----------------
def bandas_e_autovetores(malha_k: np.ndarray):
    """Diagonaliza H(k) na malha de k. Retorna E1, E2 e autovetores da banda baixa."""
    Nk = len(malha_k)
    E1 = np.zeros(Nk, dtype=float)
    E2 = np.zeros(Nk, dtype=float)
    autovec_baixa = np.zeros((Nk, 2), dtype=complex)
    for i, k in enumerate(malha_k):
        w, V = np.linalg.eigh(hamiltoniano_k(k))
        idx = np.argsort(w); w = w[idx]; V = V[:, idx]
        E1[i], E2[i] = w[0], w[1]
        v = V[:, 0]
        if np.abs(v[0]) > 1e-14:  # fixa fase: componente em C real e >=0
            v *= np.exp(-1j * np.angle(v[0]))
        autovec_baixa[i, :] = v
    return E1, E2, autovec_baixa

# ---------------- Observáveis (banda baixa ocupada) ----------------
def calcular_observaveis(autovec_baixa: np.ndarray, malha_k: np.ndarray):
    """Populações por sítio e ordens de ligação médias (2 elétrons por k)."""
    occ = 2.0
    uC = autovec_baixa[:, 0]; uN = autovec_baixa[:, 1]
    n_C = occ * np.mean(np.abs(uC)**2)
    n_N = occ * np.mean(np.abs(uN)**2)
    p_intra = occ * np.mean(np.real(np.conjugate(uC) * uN))  # C_n–N_n
    p_inter = occ * np.mean(np.real(np.exp(1j * malha_k * a) * np.conjugate(uN) * uC))  # N_n–C_{n+1}
    return float(n_C), float(n_N), float(p_intra), float(p_inter)

# ---------------- Figuras ----------------
def _fmt_matriz(M: np.ndarray) -> str:
    return f"[[{M[0,0]:6.2f}  {M[0,1]:6.2f}]\n [{M[1,0]:6.2f}  {M[1,1]:6.2f}]]"

def fig_matrizes(H_gamma: np.ndarray, H_x: np.ndarray, caminho="fig1_matrizes.png"):
    fig = plt.figure(figsize=(6.4, 4.0))
    ax = fig.add_axes([0, 0, 1, 1]); ax.axis("off")
    ax.text(0.5, 0.86, "Matrizes em Γ e X", ha="center", va="center", fontsize=13)
    ax.text(0.48, 0.62, "H(Γ) =", ha="right", va="center", fontsize=11, family="monospace")
    ax.text(0.50, 0.62, _fmt_matriz(H_gamma), ha="left", va="center", fontsize=11, family="monospace")
    ax.text(0.48, 0.38, "H(X) =", ha="right", va="center", fontsize=11, family="monospace")
    ax.text(0.50, 0.38, _fmt_matriz(H_x), ha="left", va="center", fontsize=11, family="monospace")
    ax.text(0.5, 0.14, f"Parâmetros: αC={alfa_C}, αN={alfa_N}, β={beta}, β′={beta_linha}, a={a}",
            ha="center", va="center", fontsize=9)
    fig.savefig(caminho, dpi=160); plt.close(fig)

def fig_bandas(malha_k: np.ndarray, E1: np.ndarray, E2: np.ndarray, caminho="fig2_bandas.png"):
    plt.figure(figsize=(6.4, 4.0))
    plt.plot(malha_k, E1, label="Banda 1 (baixa)")
    plt.plot(malha_k, E2, label="Banda 2 (alta)")
    plt.xlabel("k (1/a)"); plt.ylabel("Energia (eV)")
    plt.title("Estrutura de bandas – Hückel (C–N) com PBC")
    plt.xticks([-np.pi, -np.pi/2, 0.0, np.pi/2, np.pi],
               [r"$-\pi/a$", r"$-\pi/2a$", r"$\Gamma$", r"$\pi/2a$", r"$\pi/a$"])
    plt.legend(loc="best"); plt.tight_layout()
    plt.savefig(caminho, dpi=160); plt.close()

def fig_populacoes(n_C: float, n_N: float, caminho="fig3_populacoes.png"):
    plt.figure(figsize=(5.0, 3.6))
    plt.bar(["C", "N"], [n_C, n_N])
    plt.ylabel("População por célula")
    plt.title("População eletrônica (banda baixa ocupada)")
    plt.tight_layout(); plt.savefig(caminho, dpi=160); plt.close()

def fig_ordens(p_intra: float, p_inter: float, caminho="fig4_ordens.png"):
    plt.figure(figsize=(5.4, 3.6))
    plt.bar(["intra (C–N)", "inter (N–C n+1)"], [p_intra, p_inter])
    plt.ylabel("Ordem de ligação média")
    plt.title("Ordens de ligação – intra vs inter")
    plt.tight_layout(); plt.savefig(caminho, dpi=160); plt.close()

# ---------------- Execução ----------------
if __name__ == "__main__":
    H_gamma, H_x = obter_matrizes_gamma_x()
    fig_matrizes(H_gamma, H_x, "fig1_matrizes.png")

    Nk = 2001
    k = np.linspace(-np.pi / a, np.pi / a, Nk)
    E1, E2, u1 = bandas_e_autovetores(k)
    fig_bandas(k, E1, E2, "fig2_bandas.png")

    n_C, n_N, p_intra, p_inter = calcular_observaveis(u1, k)
    fig_populacoes(n_C, n_N, "fig3_populacoes.png")
    fig_ordens(p_intra, p_inter, "fig4_ordens.png")

    # Dados para reprodutibilidade
    np.savetxt("bandas.csv", np.column_stack([k, E1, E2]),
               delimiter=",", header="k,E1_baixa,E2_alta", comments="")
    with open("observaveis.json", "w", encoding="utf-8") as f:
        json.dump({"alfa_C": alfa_C, "alfa_N": alfa_N, "beta": beta, "beta_linha": beta_linha, "a": a,
                   "n_C": n_C, "n_N": n_N, "p_intra": p_intra, "p_inter": p_inter},
                  f, indent=2, ensure_ascii=False)

    # Resumo
    Eg = float(np.min(E2 - E1)); k_gap = float(k[np.argmin(E2 - E1)])
    print("H(Γ)=\n", H_gamma, "\nH(X)=\n", H_x)
    print(f"Gap mínimo: {Eg:.6f} eV em k={k_gap:.6f}")
    print(f"n_C={n_C:.6f}, n_N={n_N:.6f}, p_intra={p_intra:.6f}, p_inter={p_inter:.6f}")


H(Γ)=
 [[ 0.  -4.5]
 [-4.5 -1. ]] 
H(X)=
 [[ 0.  -0.5]
 [-0.5 -1. ]]
Gap mínimo: 1.414214 eV em k=-3.141593
n_C=0.770789, n_N=1.229211, p_intra=0.779051, p_inter=0.458089



# Relatório – Modelo de Hückel (C–N) com PBC

## 1) Matrizes H(Γ) e H(X)
Parâmetros usados: α_C=0.0 eV, α_N=-1.0 eV, β=-2.5 eV, β'=-2.0 eV, a=1.0.

![Figura 1 – Matrizes em Γ e X](matrizes.png)

*Figura 1.* Matrizes explícitas em Γ e X. Em Γ, o acoplamento efetivo é (β+β′); em X, (β−β′).

## 2) Diagonalização numérica e 3) Estrutura de bandas E(k)
Diagonalizamos H(k) para k∈[−π/a, π/a] (2001 pontos). Abaixo está o gráfico das duas bandas:

![Figura 2 – Estrutura de bandas](bandas.png)

*Figura 2.* Estrutura de bandas E₁(k) (baixa) e E₂(k) (alta). O **gap mínimo** é ≈ 1.414214 eV e ocorre em k ≈ -3.141593. Os dados numéricos estão em `bandas.csv`.

## 4) População eletrônica e ordens de ligação (banda baixa totalmente ocupada)
Assumindo a banda mais baixa totalmente ocupada (2 elétrons por k, spin):

![Figura 3 – Populações por sítio](populacoes.png)

*Figura 3.* Populações por célula: n_C = 0.770789, n_N = 1.229211. Como α_N < α_C, a banda inferior concentra maior peso no N.

![Figura 4 – Ordens de ligação](ordens_ligacao.png)

*Figura 4.* Ordens de ligação médias: intra (Cₙ–Nₙ) = 0.779051; inter (Nₙ–Cₙ₊₁) = 0.458089. Como |β| > |β′|, a ligação intra é mais forte, abrindo gap.

## 5) Código
Inclua este script no repositório GitHub e referencie o link aqui.
